## Benchmarks and Verifiers

In [1]:
import joblib
import ast
import itertools
import json
import numpy as np
from pathlib import Path
from functools import partial
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from create_feature import get_kwcounts, get_bitcounts
from parse_raw_tsv import parse_from_tsv, parse_from_tsv_lst, get_btor2path_from_yml, tsv_train_test_partition
from pairwise_model import PairwiseXGBoost, get_pw_algorithm_selection_lst
from ehm import get_ehm_algorithm_selection_lst
from analyze import get_par_N_for_dict, get_solved_num_for_dict, generate_one_resdict, generate_vbs_resdict, get_sbs
dict_par2_func = partial(get_par_N_for_dict, N=2, timeout=900.0)

In [2]:
all_res_tsv = "performance_data/performance.table.csv"
all_res_dict, tool_config_dict = parse_from_tsv(all_res_tsv, num_tool_cols=2, timeout=900.0)
tool_config_size = len(tool_config_dict)
print("Total number of benchmarks:", len(all_res_dict))
print("Total number of tools:", tool_config_size)

Total number of benchmarks: 1441
Total number of tools: 10


In [3]:
# print all components' solved numbers and par2
individual_res = []
print(f"For all {len(all_res_dict)} benchmarks (timeout=900s):")
for i in range(tool_config_size):
    one_resdict = generate_one_resdict(all_res_dict, i)
    one_par2 = get_par_N_for_dict(one_resdict, 2, 900.0)
    one_solved = get_solved_num_for_dict(one_resdict)
    individual_res.append((tool_config_dict[i][1], one_solved, one_par2))
individual_res.sort(key=lambda x: x[0])
for tool_config, solved, par2 in individual_res:
    print(f"{tool_config:<45}: solved: {solved}, par2: {par2:.1f}")

For all 1441 benchmarks (timeout=900s):
abc.bmc.bv64                                 : solved: 373, par2: 1932691.0
abc.imc.bv64                                 : solved: 690, par2: 1382558.4
abc.pdr.bv64                                 : solved: 961, par2: 909922.1
avr.bmc-boolector.bv64                       : solved: 377, par2: 1934160.3
avr.kind.bv64                                : solved: 571, par2: 1577432.5
avr.pdr.bv64                                 : solved: 784, par2: 1219703.5
cbmc.bmc.bv64                                : solved: 440, par2: 1821186.9
cbmc.kind.bv64                               : solved: 664, par2: 1431101.6
esbmc.bmc-5-boolector.bv64                   : solved: 426, par2: 1850108.2
esbmc.kind.bv64                              : solved: 515, par2: 1685649.6


In [ ]:
true_res_dict, _ = parse_from_tsv(all_res_tsv, num_tool_cols=2, timeout=900.0, verdict=True)
true_individual_res = []
print(f"For {len(true_res_dict)} benchmarks whose verdict is True (timeout=900s):")
for i in range(tool_config_size):
    one_resdict = generate_one_resdict(true_res_dict, i)
    one_par2 = get_par_N_for_dict(one_resdict, 2, 900.0)
    one_solved = get_solved_num_for_dict(one_resdict)
    true_individual_res.append((tool_config_dict[i][1], one_solved, one_par2))
true_individual_res.sort(key=lambda x: x[0])
for tool_config, solved, par2 in true_individual_res:
    print(f"{tool_config:<45}: solved: {solved}, par2: {par2:.1f}")

For 921 benchmarks whose verdict is True (timeout=900s):
abc.bmc.bv64                                 : solved: 0, par2: 1657800.0
abc.imc.bv64                                 : solved: 373, par2: 1004699.1
abc.pdr.bv64                                 : solved: 580, par2: 643305.5
avr.bmc-boolector.bv64                       : solved: 0, par2: 1657800.0
avr.kind.bv64                                : solved: 219, par2: 1265717.8
avr.pdr.bv64                                 : solved: 510, par2: 757311.8
cbmc.bmc.bv64                                : solved: 30, par2: 1603829.6
cbmc.kind.bv64                               : solved: 255, par2: 1208542.6
esbmc.bmc-5-boolector.bv64                   : solved: 43, par2: 1580436.8
esbmc.kind.bv64                              : solved: 149, par2: 1389944.0


In [6]:
false_res_dict, _ = parse_from_tsv(all_res_tsv, num_tool_cols=2, timeout=900.0, verdict=False)
false_individual_res = []
print(f"For {len(false_res_dict)} benchmarks whose verdict is False (timeout=900s):")
for i in range(tool_config_size):
    one_resdict = generate_one_resdict(false_res_dict, i)
    one_par2 = get_par_N_for_dict(one_resdict, 2, 900.0)
    one_solved = get_solved_num_for_dict(one_resdict)
    false_individual_res.append((tool_config_dict[i][1], one_solved, one_par2))
false_individual_res.sort(key=lambda x: x[0])
for tool_config, solved, par2 in false_individual_res:
    print(f"{tool_config:<45}: solved: {solved}, par2: {par2:.1f}")

For 520 benchmarks whose verdict is False (timeout=900s):
abc.bmc.bv64                                 : solved: 373, par2: 274891.0
abc.imc.bv64                                 : solved: 317, par2: 377859.3
abc.pdr.bv64                                 : solved: 381, par2: 266616.6
avr.bmc-boolector.bv64                       : solved: 377, par2: 276360.3
avr.kind.bv64                                : solved: 352, par2: 311714.7
avr.pdr.bv64                                 : solved: 274, par2: 462391.6
cbmc.bmc.bv64                                : solved: 410, par2: 217357.3
cbmc.kind.bv64                               : solved: 409, par2: 222559.0
esbmc.bmc-5-boolector.bv64                   : solved: 383, par2: 269671.3
esbmc.kind.bv64                              : solved: 366, par2: 295705.6


## Data 5-fold Split

In [11]:
# split the sets
from parse_raw_tsv import tsv_N_fold_partition
tsv_N_fold_partition('performance_data/performance.table.csv', 5, 'performance_data/5fold', random_seed=42)
res_tsv_lst = ["performance_data/5fold/0.tsv", "performance_data/5fold/1.tsv", "performance_data/5fold/2.tsv", "performance_data/5fold/3.tsv", "performance_data/5fold/4.tsv"]

/Users/zhengyanglu/opt/anaconda3/envs/btor2select/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


## EHM-BoKW
To re-train the models, run the following script:

```bash
python ehm.py --cv --save_dir=saved_models/ehm_bokw \
performance_data/5fold/0.tsv \
performance_data/5fold/1.tsv \
performance_data/5fold/2.tsv \
performance_data/5fold/3.tsv \
performance_data/5fold/4.tsv
```

In [15]:
ehm_bokw_cv_trained_model_dir = "saved_models/ehm_bokw"
ehm_bokw_cv_results = []

for i in range(len(res_tsv_lst)):
    test_res_tsv = res_tsv_lst[i]
    test_res_dict, _ = parse_from_tsv(test_res_tsv, num_tool_cols=2, timeout=900.0)
    vbs_test_resdict = generate_vbs_resdict(test_res_dict)
    vbs_test_par2 = get_par_N_for_dict(vbs_test_resdict, 2, 900.0)
    vbs_test_solved = get_solved_num_for_dict(vbs_test_resdict)
    sbs_config = get_sbs(test_res_dict, dict_par2_func)
    sbs_resdict = generate_one_resdict(test_res_dict, sbs_config)
    sbs_test_par2 = dict_par2_func(sbs_resdict)
    sbs_test_solved = get_solved_num_for_dict(sbs_resdict)

    model_dir = f"{ehm_bokw_cv_trained_model_dir}/{i}"
    ehm_regressors = []
    for j in range(tool_config_size):
        model = XGBRegressor()
        model.load_model(f"{model_dir}/xg_{j}.json")
        ehm_regressors.append(model)

    ehm_res = {}
    for benchmark in test_res_dict:
        selected_lst = get_ehm_algorithm_selection_lst(benchmark, ehm_regressors, get_kwcounts)
        selected_config = selected_lst[0]
        selected_config_name = tool_config_dict[selected_config][1]
        # print(f"Selected config {selected_config_name} for {benchmark}")
        res_tuple = test_res_dict[benchmark][selected_config]
        ehm_res[benchmark] = res_tuple

    ehm_par2 = get_par_N_for_dict(ehm_res, 2, 900.0)
    ehm_solved = get_solved_num_for_dict(ehm_res)
    par2_closed_gap = (sbs_test_par2 - ehm_par2)/(sbs_test_par2 - vbs_test_par2)*100
    solved_closed_gap = (sbs_test_solved - ehm_solved)/(sbs_test_solved - vbs_test_solved)*100
    ehm_bokw_cv_results.append((ehm_par2, ehm_solved, par2_closed_gap, solved_closed_gap))
    print(f"Experiment over {i}")
    print(f"EHM model with BoKW: Par2: {ehm_par2:.1f}, Solved: {ehm_solved}")
    print(f"Closed VBS-SBS gap: {par2_closed_gap:.2f}%")
    print(f"VBS: Par2: {vbs_test_par2:.1f}, Solved: {vbs_test_solved}")
    print(f"SBS: Par2: {sbs_test_par2:.1f}, Solved: {sbs_test_solved}")
    print()

mean_ehm_bokw_par2 = np.mean([x[0] for x in ehm_bokw_cv_results])
mean_ehm_bokw_solved = np.mean([x[1] for x in ehm_bokw_cv_results])
mean_par2_gap = np.mean([x[2] for x in ehm_bokw_cv_results])
mean_solved_gap = np.mean([x[3] for x in ehm_bokw_cv_results])
print("Mean over 5 folds")
print(f"EHM model with BoKW: Par2: {mean_ehm_bokw_par2:.1f}, Solved: {mean_ehm_bokw_solved}, Par2 gap: {mean_par2_gap:.2f}%, Solved gap: {mean_solved_gap:.2f}%")

Experiment over 0
EHM model with BoKW: Par2: 155916.4, Solved: 208
Closed VBS-SBS gap: 46.88%
VBS: Par2: 121767.9, Solved: 227
SBS: Par2: 186053.3, Solved: 191

Experiment over 1
EHM model with BoKW: Par2: 152401.3, Solved: 208
Closed VBS-SBS gap: 47.78%
VBS: Par2: 119870.9, Solved: 227
SBS: Par2: 182165.2, Solved: 190

Experiment over 2
EHM model with BoKW: Par2: 157530.9, Solved: 206
Closed VBS-SBS gap: 62.24%
VBS: Par2: 132034.0, Solved: 220
SBS: Par2: 199555.3, Solved: 183

Experiment over 3
EHM model with BoKW: Par2: 148497.5, Solved: 210
Closed VBS-SBS gap: 56.02%
VBS: Par2: 127329.0, Solved: 222
SBS: Par2: 175465.0, Solved: 196

Experiment over 4
EHM model with BoKW: Par2: 123815.0, Solved: 226
Closed VBS-SBS gap: 61.19%
VBS: Par2: 96629.8, Solved: 239
SBS: Par2: 166683.4, Solved: 201

Mean over 5 folds
EHM model with BoKW: Par2: 147632.2, Solved: 211.6, Par2 gap: 54.82%, Solved gap: 55.53%


## EHM-BWA
To re-train the models, run the following script:

```bash
python ehm.py --cv --save_dir=saved_models/ehm_bwa \
performance_data/5fold/0.tsv \
performance_data/5fold/1.tsv \
performance_data/5fold/2.tsv \
performance_data/5fold/3.tsv \
performance_data/5fold/4.tsv
```

In [17]:
ehm_bwa_cv_trained_model_dir = "saved_models/ehm_bwa"
ehm_bwa_cv_results = []

for i in range(len(res_tsv_lst)):
    test_res_tsv = res_tsv_lst[i]
    test_res_dict, _ = parse_from_tsv(test_res_tsv, num_tool_cols=2, timeout=900.0)
    vbs_test_resdict = generate_vbs_resdict(test_res_dict)
    vbs_test_par2 = get_par_N_for_dict(vbs_test_resdict, 2, 900.0)
    vbs_test_solved = get_solved_num_for_dict(vbs_test_resdict)
    sbs_config = get_sbs(test_res_dict, dict_par2_func)
    sbs_resdict = generate_one_resdict(test_res_dict, sbs_config)
    sbs_test_par2 = dict_par2_func(sbs_resdict)
    sbs_test_solved = get_solved_num_for_dict(sbs_resdict)

    model_dir = f"{ehm_bwa_cv_trained_model_dir}/{i}"
    ehm_regressors = []
    for j in range(tool_config_size):
        model = XGBRegressor()
        model.load_model(f"{model_dir}/xg_{j}.json")
        ehm_regressors.append(model)

    ehm_res = {}
    for benchmark in test_res_dict:
        selected_lst = get_ehm_algorithm_selection_lst(benchmark, ehm_regressors, get_bitcounts)
        selected_config = selected_lst[0]
        selected_config_name = tool_config_dict[selected_config][1]
        # print(f"Selected config {selected_config_name} for {benchmark}")
        res_tuple = test_res_dict[benchmark][selected_config]
        ehm_res[benchmark] = res_tuple

    ehm_par2 = get_par_N_for_dict(ehm_res, 2, 900.0)
    ehm_solved = get_solved_num_for_dict(ehm_res)
    par2_closed_gap = (sbs_test_par2 - ehm_par2)/(sbs_test_par2 - vbs_test_par2)*100
    solved_closed_gap = (sbs_test_solved - ehm_solved)/(sbs_test_solved - vbs_test_solved)*100
    ehm_bwa_cv_results.append((ehm_par2, ehm_solved, par2_closed_gap, solved_closed_gap))
    print(f"Experiment over {i}")
    print(f"EHM model with BWA: Par2: {ehm_par2:.1f}, Solved {ehm_solved}")
    print(f"Closed VBS-SBS gap: {par2_closed_gap:.2f}%")
    print(f"VBS: Par2: {vbs_test_par2:.1f}, Solved: {vbs_test_solved}")
    print(f"SBS: Par2: {sbs_test_par2:.1f}, Solved: {sbs_test_solved}")
    print()

mean_ehm_bwa_par2 = np.mean([x[0] for x in ehm_bwa_cv_results])
mean_ehm_bwa_solved = np.mean([x[1] for x in ehm_bwa_cv_results])
mean_par2_gap = np.mean([x[2] for x in ehm_bwa_cv_results])
mean_solved_gap = np.mean([x[3] for x in ehm_bwa_cv_results])
print("Mean over 5 folds")
print(f"EHM model with BWA: Par2: {mean_ehm_bwa_par2:.1f}, Solved: {mean_ehm_bwa_solved}, Par2 gap: {mean_par2_gap:.2f}%, Solved gap: {mean_solved_gap:.2f}%")

Experiment over 0
EHM model with BWA: Par2: 152782.2, Solved 210
Closed VBS-SBS gap: 51.76%
VBS: Par2: 121767.9, Solved: 227
SBS: Par2: 186053.3, Solved: 191

Experiment over 1
EHM model with BWA: Par2: 140190.2, Solved 215
Closed VBS-SBS gap: 67.38%
VBS: Par2: 119870.9, Solved: 227
SBS: Par2: 182165.2, Solved: 190

Experiment over 2
EHM model with BWA: Par2: 165850.1, Solved 201
Closed VBS-SBS gap: 49.92%
VBS: Par2: 132034.0, Solved: 220
SBS: Par2: 199555.3, Solved: 183

Experiment over 3
EHM model with BWA: Par2: 144904.0, Solved 213
Closed VBS-SBS gap: 63.49%
VBS: Par2: 127329.0, Solved: 222
SBS: Par2: 175465.0, Solved: 196

Experiment over 4
EHM model with BWA: Par2: 128188.4, Solved 223
Closed VBS-SBS gap: 54.95%
VBS: Par2: 96629.8, Solved: 239
SBS: Par2: 166683.4, Solved: 201

Mean over 5 folds
EHM model with BWA: Par2: 146383.0, Solved: 212.4, Par2 gap: 57.50%, Solved gap: 58.45%


## PWC-BoKW

To re-train the models, run the following script:

```bash
python pairwise_model.py --cv --save_dir=saved_models/pw_bokw \
performance_data/5fold/0.tsv \
performance_data/5fold/1.tsv \
performance_data/5fold/2.tsv \
performance_data/5fold/3.tsv \
performance_data/5fold/4.tsv
```

In [12]:
pw_bokw_cv_trained_model_dir = "saved_models/pw_bokw"
pw_bokw_cv_results = []

for cv_i in range(len(res_tsv_lst)):
    print(f"Cross validation fold {cv_i}")
    test_res_tsv = res_tsv_lst[cv_i]
    test_res_dict, _ = parse_from_tsv(test_res_tsv, num_tool_cols=2, timeout=900.0)
    vbs_test_resdict = generate_vbs_resdict(test_res_dict)
    vbs_test_par2 = get_par_N_for_dict(vbs_test_resdict, 2, 900.0)
    vbs_test_solved = get_solved_num_for_dict(vbs_test_resdict)
    sbs_config = get_sbs(test_res_dict, dict_par2_func)
    sbs_resdict = generate_one_resdict(test_res_dict, sbs_config)
    sbs_test_par2 = dict_par2_func(sbs_resdict)
    sbs_test_solved = get_solved_num_for_dict(sbs_resdict)

    model_dir = f"{pw_bokw_cv_trained_model_dir}/{cv_i}"
    xg_matrix = np.empty((tool_config_size, tool_config_size), dtype=object)
    xg_matrix[:] = None
    for i in range(tool_config_size):
        for j in range(i+1, tool_config_size):
            model_path = Path(model_dir) / f"xg_{i}_{j}.joblib"
            xg_matrix[i, j] = joblib.load(model_path)
            
    select_res_dict = {}
    for test_benchmark in test_res_dict:
        selected_lst = get_pw_algorithm_selection_lst(test_benchmark, xg_matrix, get_kwcounts)
        selected_config = selected_lst[0]
        selected_config_name = tool_config_dict[selected_config][1]
        res_tuple = test_res_dict[test_benchmark][selected_config]
        select_res_dict[test_benchmark] = res_tuple

    select_par2 = get_par_N_for_dict(select_res_dict, 2, 900.0)
    select_solved = get_solved_num_for_dict(select_res_dict)
    par2_closed = (sbs_test_par2 - select_par2)/(sbs_test_par2 - vbs_test_par2)*100
    solved_closed = (sbs_test_solved - select_solved)/(sbs_test_solved - vbs_test_solved)*100
    pw_bokw_cv_results.append((select_par2, select_solved, par2_closed, solved_closed))
    print(f"Pairwise model with BoWA: Par2: {select_par2:.1f}, Solved: {select_solved}")
    print(f"Closed VBS-SBS gap: {par2_closed:.2f}%\n")

mean_pw_bwa_par2 = np.mean([res[0] for res in pw_bokw_cv_results])
mean_pw_bwa_solved = np.mean([res[1] for res in pw_bokw_cv_results])
mean_par2_closed = np.mean([res[2] for res in pw_bokw_cv_results])
mean_solved_closed = np.mean([res[3] for res in pw_bokw_cv_results])
print(f"Mean over 5 folds")
print(f"Pairwise model with BoKW: Par2: {mean_pw_bwa_par2:.1f}, Solved: {mean_pw_bwa_solved}")
print(f"Closed VBS-SBS gap: {mean_par2_closed:.2f}% in par2 and {mean_solved_closed:.2f}% in solved\n")

Cross validation fold 0
Pairwise model with BoWA: Par2: 143824.2, Solved: 214
Closed VBS-SBS gap: 65.69%

Cross validation fold 1
Pairwise model with BoWA: Par2: 142076.4, Solved: 213
Closed VBS-SBS gap: 64.35%

Cross validation fold 2
Pairwise model with BoWA: Par2: 156107.1, Solved: 206
Closed VBS-SBS gap: 64.35%

Cross validation fold 3
Pairwise model with BoWA: Par2: 140596.1, Solved: 215
Closed VBS-SBS gap: 72.44%

Cross validation fold 4
Pairwise model with BoWA: Par2: 120484.8, Solved: 227
Closed VBS-SBS gap: 65.95%

Mean over 5 folds
Pairwise model with BoKW: Par2: 140617.7, Solved: 215.0
Closed VBS-SBS gap: 66.56% in par2 and 65.94% in solved



## PWC-BWA

To re-train the models, run the following script:

```bash
python pairwise_model.py --bit --cv --save_dir=saved_models/pw_bwa \
performance_data/5fold/0.tsv \
performance_data/5fold/1.tsv \
performance_data/5fold/2.tsv \
performance_data/5fold/3.tsv \
performance_data/5fold/4.tsv
```

In [16]:
pw_bwa_cv_trained_model_dir = "saved_models/pw_bwa"
pw_bwa_cv_results = []

for cv_i in range(len(res_tsv_lst)):
    print(f"Cross validation fold {cv_i}")
    test_res_tsv = res_tsv_lst[cv_i]
    test_res_dict, _ = parse_from_tsv(test_res_tsv, num_tool_cols=2, timeout=900.0)
    vbs_test_resdict = generate_vbs_resdict(test_res_dict)
    vbs_test_par2 = get_par_N_for_dict(vbs_test_resdict, 2, 900.0)
    vbs_test_solved = get_solved_num_for_dict(vbs_test_resdict)
    sbs_config = get_sbs(test_res_dict, dict_par2_func)
    sbs_resdict = generate_one_resdict(test_res_dict, sbs_config)
    sbs_test_par2 = dict_par2_func(sbs_resdict)
    sbs_test_solved = get_solved_num_for_dict(sbs_resdict)

    model_dir = f"{pw_bwa_cv_trained_model_dir}/{cv_i}"
    xg_matrix = np.empty((tool_config_size, tool_config_size), dtype=object)
    xg_matrix[:] = None
    for i in range(tool_config_size):
        for j in range(i+1, tool_config_size):
            model_path = Path(model_dir) / f"xg_{i}_{j}.joblib"
            xg_matrix[i, j] = joblib.load(model_path)
            
    select_res_dict = {}
    for test_benchmark in test_res_dict:
        selected_lst = get_pw_algorithm_selection_lst(test_benchmark, xg_matrix, get_bitcounts)
        selected_config = selected_lst[0]
        selected_config_name = tool_config_dict[selected_config][1]
        res_tuple = test_res_dict[test_benchmark][selected_config]
        select_res_dict[test_benchmark] = res_tuple

    select_par2 = get_par_N_for_dict(select_res_dict, 2, 900.0)
    select_solved = get_solved_num_for_dict(select_res_dict)
    par2_closed = (sbs_test_par2 - select_par2)/(sbs_test_par2 - vbs_test_par2)*100
    solved_closed = (sbs_test_solved - select_solved)/(sbs_test_solved - vbs_test_solved)*100
    pw_bwa_cv_results.append((select_par2, select_solved, par2_closed, solved_closed))
    print(f"Pairwise model with BWA: Par2: {select_par2:.1f}, Solved: {select_solved}")
    print(f"Closed VBS-SBS gap: {par2_closed:.2f}%\n")

mean_pw_bwa_par2 = np.mean([res[0] for res in pw_bwa_cv_results])
mean_pw_bwa_solved = np.mean([res[1] for res in pw_bwa_cv_results])
mean_par2_closed = np.mean([res[2] for res in pw_bwa_cv_results])
mean_solved_closed = np.mean([res[3] for res in pw_bwa_cv_results])
print(f"Mean over 5 folds")
print(f"Pairwise model with BWA: Par2: {mean_pw_bwa_par2:.1f}, Solved: {mean_pw_bwa_solved}")
print(f"Closed VBS-SBS gap: {mean_par2_closed:.2f}% in par2 and {mean_solved_closed:.2f}% in solved\n")


Cross validation fold 0
Pairwise model with BWA: Par2: 148966.1, Solved: 211
Closed VBS-SBS gap: 57.69%

Cross validation fold 1
Pairwise model with BWA: Par2: 145743.9, Solved: 211
Closed VBS-SBS gap: 58.47%

Cross validation fold 2
Pairwise model with BWA: Par2: 158159.3, Solved: 205
Closed VBS-SBS gap: 61.31%

Cross validation fold 3
Pairwise model with BWA: Par2: 136560.5, Solved: 217
Closed VBS-SBS gap: 80.82%

Cross validation fold 4
Pairwise model with BWA: Par2: 122008.9, Solved: 226
Closed VBS-SBS gap: 63.77%

Mean over 5 folds
Pairwise model with BWA: Par2: 142287.7, Solved: 214.0
Closed VBS-SBS gap: 64.41% in par2 and 63.67% in solved



## DQN-BoKW

To re-create the BoKW feature database: `benchmarks/bv/bv_btor2kw.csv`

```bash
python create_feature.py benchmarks/bv benchmarks/bv/bv_btor2kw.csv
```

To re-train the DQN model, an example for the model to be tested on 0.tsv is shown as below:

```bash
python train_dqn.py -m=saved_models/dqn/0 \
performance_data/5fold/1.tsv \
performance_data/5fold/2.tsv \
performance_data/5fold/3.tsv \
performance_data/5fold/4.tsv
```

In [2]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import DQN
from torch import tensor

from parse_raw_tsv import parse_from_tsv
from create_feature import KEYWORDS, read_btor2kw_csv
from btor2select_env import Btor2SelectMDP_D_test
from train_dqn import add_scaled_kw

In [3]:
class MaskedTestDQN(DQN):
    def __init__(self, *args, **kwargs):
        super(MaskedTestDQN, self).__init__(*args, **kwargs)

    def masked_predict(self, obs, masks):
        if len(obs.shape) == 1:
            obs = np.expand_dims(obs, axis=0)
        q_values = self.policy.q_net(tensor(obs).float().to(self.device))
        q_values = q_values.cpu().detach().numpy()
        masks = masks.astype(int)
        for i in range(len(obs)):
            q_values[i][masks[i] == 0] = -np.inf
        actions = q_values.argmax(axis=1)
        return actions

In [5]:
kw_size = len(KEYWORDS)
kw_embds = read_btor2kw_csv("benchmarks/bv/bv_btor2kw.csv")
sc_kw_embds, kw_scaler = add_scaled_kw(kw_embds)

Cross Validation for 0.tsv

In [12]:
test_res_tsv0 = "performance_data/5fold/0.tsv"
test_res_dict0, tool_config_dict0 = parse_from_tsv(test_res_tsv0, num_tool_cols=2, timeout=900.0)
test_env0 = Btor2SelectMDP_D_test(
                instance_embed_size=kw_size,
                res_dict=test_res_dict0,
                tool_config_dict=tool_config_dict0,
                early_timeouts=[100, 300],
                embd_dict=sc_kw_embds,
                timeout=900
            )
agent0 = MaskedTestDQN.load("saved_models/dqn/0/best_model.zip")
total_test_size0 = len(test_res_dict0)
num_solved0 = 0
par2_0 = 0
test_results0 = {}
obs, reset_info = test_env0.reset_test()
while not test_env0.is_test_done():
    masks = np.array([test_env0.get_legal_actions()])
    actions = agent0.masked_predict(obs, masks)
    action = actions[0]
    obs, _, eps_terminated, _, info = test_env0.step(action)
    if eps_terminated:
        if info["is_success"]:
            num_solved0 += 1
            par2_0 += info["time"]
        else:
            par2_0 += 1800
        obs, reset_info = test_env0.reset_test()
print(f"Success rate: {num_solved0}/{total_test_size0} = {num_solved0/total_test_size0:.2f}")
print(f"PAR-2: {par2_0:.2f}")

Success rate: 215/289 = 0.74
PAR-2: 143949.02


Cross Validation for 1.tsv

In [13]:
test_res_tsv1 = "performance_data/5fold/1.tsv"
test_res_dict1, tool_config_dict1 = parse_from_tsv(test_res_tsv1, num_tool_cols=2, timeout=900.0)
test_env1 = Btor2SelectMDP_D_test(
                instance_embed_size=kw_size,
                res_dict=test_res_dict1,
                tool_config_dict=tool_config_dict1,
                early_timeouts=[100, 300],
                embd_dict=sc_kw_embds,
                timeout=900
            )
agent1 = MaskedTestDQN.load("saved_models/dqn/1/best_model.zip")
total_test_size1 = len(test_res_dict1)
num_solved1 = 0
par2_1 = 0
test_results1 = {}
obs, reset_info = test_env1.reset_test()
while not test_env1.is_test_done():
    masks = np.array([test_env1.get_legal_actions()])
    actions = agent1.masked_predict(obs, masks)
    action = actions[0]
    obs, _, eps_terminated, _, info = test_env1.step(action)
    if eps_terminated:
        if info["is_success"]:
            num_solved1 += 1
            par2_1 += info["time"]
        else:
            par2_1 += 1800
        obs, reset_info = test_env1.reset_test()
print(f"Success rate: {num_solved1}/{total_test_size1} = {num_solved1/total_test_size1:.2f}")
print(f"PAR-2: {par2_1:.2f}")

Success rate: 216/288 = 0.75
PAR-2: 141236.71


/Users/zhengyanglu/opt/anaconda3/envs/btor2select/lib/python3.12/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 0.66GB > 0.37GB
  warnings.warn(


Cross Validation for 2.tsv

In [14]:
test_res_tsv2 = "performance_data/5fold/2.tsv"
test_res_dict2, tool_config_dict2 = parse_from_tsv(test_res_tsv2, num_tool_cols=2, timeout=900.0)
test_env2 = Btor2SelectMDP_D_test(
                instance_embed_size=kw_size,
                res_dict=test_res_dict2,
                tool_config_dict=tool_config_dict2,
                early_timeouts=[100, 300],
                embd_dict=sc_kw_embds,
                timeout=900
            )
agent2 = MaskedTestDQN.load("saved_models/dqn/2/best_model.zip")
total_test_size2 = len(test_res_dict2)
num_solved2 = 0
par2_2 = 0

test_results2 = {}
obs, reset_info = test_env2.reset_test()
while not test_env2.is_test_done():
    masks = np.array([test_env2.get_legal_actions()])
    actions = agent2.masked_predict(obs, masks)
    action = actions[0]
    obs, _, eps_terminated, _, info = test_env2.step(action)
    if eps_terminated:
        if info["is_success"]:
            num_solved2 += 1
            par2_2 += info["time"]
        else:
            par2_2 += 1800
        obs, reset_info = test_env2.reset_test()
print(f"Success rate: {num_solved2}/{total_test_size2} = {num_solved2/total_test_size2:.2f}")
print(f"PAR-2: {par2_2:.2f}")

Success rate: 214/288 = 0.74
PAR-2: 145255.62


Cross Validation for 3.tsv

In [15]:
test_res_tsv3 = "performance_data/5fold/3.tsv"
test_res_dict3, tool_config_dict3 = parse_from_tsv(test_res_tsv3, num_tool_cols=2, timeout=900.0)
test_env3 = Btor2SelectMDP_D_test(
                instance_embed_size=kw_size,
                res_dict=test_res_dict3,
                tool_config_dict=tool_config_dict3,
                early_timeouts=[100, 300],
                embd_dict=sc_kw_embds,
                timeout=900
            )
agent3 = MaskedTestDQN.load("saved_models/dqn/3/best_model.zip")
total_test_size3 = len(test_res_dict3)
num_solved3 = 0
par2_3 = 0
test_results3 = {}
obs, reset_info = test_env3.reset_test()
while not test_env3.is_test_done():
    masks = np.array([test_env3.get_legal_actions()])
    actions = agent3.masked_predict(obs, masks)
    action = actions[0]
    obs, _, eps_terminated, _, info = test_env3.step(action)
    if eps_terminated:
        if info["is_success"]:
            num_solved3 += 1
            par2_3 += info["time"]
        else:
            par2_3 += 1800
        obs, reset_info = test_env3.reset_test()
print(f"Success rate: {num_solved3}/{total_test_size3} = {num_solved3/total_test_size3:.2f}")
print(f"PAR-2: {par2_3:.2f}")

Success rate: 216/288 = 0.75
PAR-2: 137081.86


Cross Validation for 4.tsv

In [16]:
test_res_tsv4 = "performance_data/5fold/4.tsv"
test_res_dict4, tool_config_dict4 = parse_from_tsv(test_res_tsv4, num_tool_cols=2, timeout=900.0)
test_env4 = Btor2SelectMDP_D_test(
                instance_embed_size=kw_size,
                res_dict=test_res_dict4,
                tool_config_dict=tool_config_dict4,
                early_timeouts=[100, 300],
                embd_dict=sc_kw_embds,
                timeout=900
            )
agent4 = MaskedTestDQN.load("saved_models/dqn/4/best_model.zip")
total_test_size4 = len(test_res_dict4)
num_solved4 = 0
par2_4 = 0
test_results4 = {}
obs, reset_info = test_env4.reset_test()
while not test_env4.is_test_done():
    masks = np.array([test_env4.get_legal_actions()])
    actions = agent4.masked_predict(obs, masks)
    action = actions[0]
    obs, _, eps_terminated, _, info = test_env4.step(action)
    if eps_terminated:
        if info["is_success"]:
            num_solved4 += 1
            par2_4 += info["time"]
        else:
            par2_4 += 1800
        obs, reset_info = test_env4.reset_test()
print(f"Success rate: {num_solved4}/{total_test_size4} = {num_solved4/total_test_size4:.2f}")
print(f"PAR-2: {par2_4:.2f}")

Success rate: 226/288 = 0.78
PAR-2: 120361.57


/Users/zhengyanglu/opt/anaconda3/envs/btor2select/lib/python3.12/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 0.66GB > 0.36GB
  warnings.warn(


Average 5-fold

In [18]:
average_solved = (num_solved0 + num_solved1 + num_solved2 + num_solved3 + num_solved4) / 5
average_par2 = (par2_0 + par2_1 + par2_2 + par2_3 + par2_4) / 5
print(f"Average solved: {average_solved:.1f}")
print(f"Average PAR-2: {average_par2:.1f}")
ave_vbs_par2 = 119526.3
ave_sbs_par2 = 181984.4
print(f"closed par-2: {(average_par2 - ave_sbs_par2) / (ave_vbs_par2 - ave_sbs_par2):.5f}")

Average solved: 217.4
Average PAR-2: 137577.0
closed par-2: 0.71100


## Shapley Contribution
See `reproduce_aaai.ipynb` for the computation of Shapley contributions.